# Step 1: annotazione

Frame OLIVERio:
[ ] Concessive
[ ] History
[ ] Change_resistance
[ ] Emptying
[ ] Performers_and_roles

Frame TOMATIS
[x] Deciding
[x] Intentionally_act
[x] Competition
[x] Fairness_evaluation
[x] Process_continue

In [80]:
from nltk.corpus import framenet as fn
from nltk.corpus import wordnet as wn

frame = fn.frame_by_name('Concessive')

print('definizione frame:', frame.definition)
print('\n')

for fe in frame.FE:
    print(fe, ': ', frame.FE[fe].definition)

print('\n')
for lu in frame.lexUnit:
    print(lu, ': ', frame.lexUnit[lu].definition)

definizione frame: A speaker marks a Conceded_state_of_affairs as being or implying a contradiction to the Main_assertion.  Some LUs specify the Main_assertion, others the Conceded_state_of_affairs, as referring to the preceding discourse (i.e. DNI), while yet others are conjunctions joining the two.  'Although she had some trouble , she was able to finish about two hours early .' 'Nevertheless , we did win . DNI ' 'I wanted to come , but I did n't remember where the party was . '


frame element (13302): Conceded_state_of_affairs
    of Concessive(2150)
[definition]
  A proposition or set of propositions that the speaker wants the
  addressee to believe or take for granted. This statement is not
  asserted, but presupposed by the speaker of the sentence, and is
  mentioned to contrast with the Main_assertion. It would or could
  seem to predict a situation different than the Main_assertion,
  but, for reasons left implicit, this normal prediction is
  contradicted by the fact that the

In [81]:
fn.lu(256)

lexical unit (256): foresee.v

[definition]
  COD: be aware of beforehand; predict.

[frame] Expectation(26)

[POS] V

[status] FN1_Sent

[lexemes] foresee/V

[semTypes] 0 semantic types

[URL] https://framenet2.icsi.berkeley.edu/fnReports/data/lu/lu256.xml

[subCorpus] 25 subcorpora
  V-420-that-sfin, V-430-sfin, V-480-swh, V-570-np-ppfor,
  V-570-np-ppfrom, V-570-np-ppin, V-570-np-ppon, V-570-np-
  ppwith, V-620-np-ppother, V-650-np-pother, V-660-trans-
  simple, V-670-pass-by, V-680-pass, V-690-trans-other,
  V-730-ppat, V-730-ppfor, V-730-ppfrom, V-730-ppin,
  V-730-ppon, V-730-ppwith, V-780-ppother, V-810-pother,
  V-890-intrans-adverb, V-900-other, manually-added

[exemplars] 45 sentences across all subcorpora

In [78]:
print('\n')
for syn in wn.synsets('deed'):
    print(syn)
    print('definition:' , syn.definition())
    #print example
    for example in syn.examples():
        print(example)

    print('\n')



Synset('deed.n.01')
definition: a legal document signed and sealed and delivered to effect a transfer of property and to show the legal right to possess it
he signed the deed
he kept the title to his car in the glove compartment


Synset('act.n.02')
definition: something that people do or cause to happen




# Step 2: mapping automatico

## Approccio bag of words

In [45]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer #tiene conto delle multiword expressions
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
import re


In [46]:
deciding_frame = fn.frame_by_name("Deciding")

In [47]:
#creazione tokenizzatore per le multiword expressions
mwes = [x for x in wn.all_lemma_names() if '_' in x]
mwes = [tuple(x.split('_')) for x in mwes]
tokenizer = MWETokenizer(mwes, separator=' ')

def make_set(sentence):
    sentence = sentence.lower() #lowercase
    sentence = re.sub(r'[^\w\s]','',sentence) #remove punctuation
    sentence = tokenizer.tokenize(sentence.split()) #tokenize
    sentence = [w for w in sentence if not w.isdigit()] #remove numbers
    stop_words = set(stopwords.words('english')) #remove stop words
    sentence = [w for w in sentence if not w in stop_words]
    lemmatizer = WordNetLemmatizer() #lemmatization of definition
    sentence = [lemmatizer.lemmatize(w) for w in sentence]

    res = []
    for w in sentence:
        res.append(w.replace(' ', '_'))

    return set(res)

make_set("I'm going to the park of New York.")

{'going', 'im', 'new_york', 'park'}

In [48]:
def ctx_frame_FN(frame):
    name = frame.name
    definition = frame.definition
    FEs = frame.FE
    LUs = frame.lexUnit

    sentence = name + ' ' + definition
    for fe in FEs:
        sentence += ' ' + FEs[fe].definition
    for lu in LUs:
        sentence += ' ' + LUs[lu].definition

    return make_set(sentence)

def ctx_frame_element_FN(frame_element):
    name = frame_element.name
    definition = frame_element.definition
    #semtype = frame_element.semType.name

    sentence = name + ' ' + definition # + ' ' + semtype
   
    return make_set(sentence)

def ctx_lexical_unit_FN(lexical_unit):
    name = lexical_unit.name
    definition = lexical_unit.definition.split(':')[1]
    exemplars = lexical_unit.exemplars
    
    sentence = name + ' ' + definition
    for ex in exemplars:
        sentence += ' ' + ex.annotationSet[0].text

    return make_set(sentence)


### Creazione del contesto (FrameNet)

In [49]:
ctx_FN = {}

#frame
ctx_FN[deciding_frame.name.replace(' ', '_')] = ctx_frame_FN(deciding_frame)

#FEs
for fe in deciding_frame.FE:
    ctx_FN[fe.replace(' ', '_')] = ctx_frame_element_FN(deciding_frame.FE[fe])
    break

#LUs
for lu in deciding_frame.lexUnit:
    ctx_FN[lu.replace(' ', '_')] = ctx_lexical_unit_FN(deciding_frame.lexUnit[lu])


### Crezione del contesto (WordNet)

In [50]:
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= set(get_hyponyms(hyponym))
    return hyponyms | set(synset.hyponyms())

In [51]:
def create_sentences(synset, depth=1):
    sentence = synset.definition()
    for example in synset.examples():
        sentence += ' ' + example
    for lemma in synset.lemmas():
        sentence += ' ' + lemma.name()

    if (depth >= 0):
        for hypernym in synset.hypernyms():
            sentence += ' ' + create_sentences(hypernym, depth-1)
        for hyponym in get_hyponyms(synset):
            sentence += ' ' + create_sentences(hyponym, depth-1)  

    return sentence


def ctx_synset_WN(synset):
    sentence = create_sentences(synset)
    return make_set(sentence)

In [52]:
mappings = {}
for key in ctx_FN:
    token = key.split('.')[0]
    syns = wn.synsets(token)
    max_overlap = 0
    for syn in syns:
        overlap = len(ctx_synset_WN(syn).intersection(ctx_FN[key])) + 1
        if overlap > max_overlap:
            max_overlap = overlap
            mappings[key] = syn

mappings

{'Deciding': Synset('decision_making.n.01'),
 'decide.v': Synset('decide.v.02'),
 'decision.n': Synset('decision.n.01'),
 'rule_out.v': Synset('rule_out.v.02'),
 'determine.v': Synset('determine.v.02')}

## Approccio grafico

In [53]:
deciding_frame = fn.frame_by_name("Deciding")

In [54]:
#FEs
for fe in deciding_frame.FE: 
    ctx_FN[fe.replace(' ', '_')] = ctx_frame_element_FN(deciding_frame.FE[fe])
#LUs
for lu in deciding_frame.lexUnit:
    ctx_FN[lu.replace(' ', '_')] = ctx_lexical_unit_FN(deciding_frame.lexUnit[lu])

Cognizer
Decision
Possibilities
Time
Place
Manner
Inherent_purpose
Circumstance
Explanation


In [55]:
def get_paths_between_synsets(synset1, synset2, L=3):
    paths = []
    visited = set()

    def dfs(synset, path):
        if synset in visited or len(path) > L:
            return
        if synset == synset2:
            paths.append(path + [synset])
            return
        visited.add(synset)
        for hypernym in synset.hypernyms():
            dfs(hypernym, path + [synset])
        for hyponym in synset.hyponyms():
            dfs(hyponym, path + [synset])

    dfs(synset1, [])
    return [path for path in paths if len(path) <= L]

In [56]:
import numpy as np

def score(syn_FN, word_FN, ctx_FN):
    ctx_word = ctx_FN[word_FN]

    res = 0
    for word in ctx_word:
        for syn in wn.synsets(word):
            #get all connection path between syn and syn_FN
            paths = list(get_paths_between_synsets(syn, syn_FN, 3))

            for path in paths:
                res += np.exp(-len(path)-1)

    return res

def prob(syn_FN, word_FN, ctx_FN):
    sum = 0
    for key in ctx_FN:
        for syn in wn.synsets(key):
            sum += score(syn, key, ctx_FN)

    return score(syn_FN, word_FN, ctx_FN) / sum

def argmax_prob(word_FN, ctx_FN):
    max_prob = 0
    max_syn = None
    for syn in wn.synsets(word_FN):
        prob_syn = prob(syn, word_FN, ctx_FN)
        if prob_syn > max_prob:
            max_prob = prob_syn
            max_syn = syn
            
    return max_syn

print(wn.synsets('deciding'))

argmax_prob('Deciding', ctx_FN)

[Synset('decision_making.n.01'), Synset('decide.v.01'), Synset('decide.v.02'), Synset('decide.v.03'), Synset('decide.v.04'), Synset('deciding.s.01')]


Synset('decide.v.01')

# Step 3: valutazione

In [ ]:
# to do...